# Set up the environment

In [8]:
import pandas as pd
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
from src.plotting import trim_axs
from tqdm import tqdm
from pathlib import Path

In [9]:
import os
os.environ["SUBJECTS_DIR"] = "/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer"
os.environ["FREESURFER_HOME"] = "/Applications/freesurfer"

In [10]:
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling'
process = 'WholeBrainPlotting'
interim_dir = f'{top_dir}/data/interim'
raw_dir = f'{top_dir}/data/raw'
figure_dir = f'{top_dir}/reports/figures'
Path(f'{interim_dir}/{process}').mkdir(exist_ok=True, parents=True)
Path(f'{figure_dir}/{process}').mkdir(exist_ok=True, parents=True)

In [11]:
metadata = pd.read_csv('../data/interim/ReorganziefMRI/metadata.csv')

In [20]:
path = '../data/interim/ModelTopPerforming'
files = {'video': f'{path}/VideoNeuralEncoding_None.csv.gz',
         'language': f'{path}/LanguageNeuralEncoding_none.csv.gz',
         }

# Surf plotting functions

In [ ]:
import numpy as np
from nilearn import plotting, surface
import nibabel as nib
from src.tools import camera_switcher


def compute_surf_stats(prefix, sub, hemi):
    file = f'{prefix}_hemi-{hemi}.mgz'
    if not os.path.exists(file):
        cmd = '/Applications/freesurfer/bin/mri_vol2surf '
        cmd += f'--src {prefix}.nii.gz '
        cmd += f'--out {file} '
        cmd += f'--regheader sub-{sub} '
        cmd += f'--hemi {hemi} '
        cmd += '--projfrac 1'
        os.system(cmd)
    return surface.load_surf_data(file)


def load_surf_mesh(path, sub, hemi):
    return f'{path}/freesurfer/sub-{sub}/surf/{hemi}.inflated', \
            f'{path}/freesurfer/sub-{sub}/surf/{hemi}.sulc'


def plot_stats(surf_mesh, bg_map, surf_map, hemi_, figure_prefix,
                vmax=0.6, negative_values=False,
                cmap_name='icefire', threshold=1e-6):
    cmap=sns.color_palette(cmap_name, as_cmap=True)
    hemi_name = 'left' if hemi_ == 'lh' else 'right'

    surf_map = np.nan_to_num(surf_map)
    if not negative_values:
        surf_map[surf_map < 0] = 0
        vmin = 0
        tickvals = list(np.round(np.linspace(vmin, vmax, num=5), 2)) 
    else:
        vmin = None
        tickvals = list(np.round(np.linspace(-1*vmax, vmax, num=5), 2)) 

    for view in ['ventral', 'lateral', 'medial']:
        colorbar = True if view == 'medial' and hemi_ == 'rh' else False
        fig = plotting.plot_surf_roi(surf_mesh=surf_mesh,
                                        roi_map=surf_map,
                                        bg_map=bg_map,
                                        threshold=threshold,
                                        vmax=vmax,
                                        vmin=vmin,
                                        engine='plotly',
                                        colorbar=colorbar,
                                        view=view,
                                        cmap=cmap,
                                        hemi=hemi_name)
        fig.figure.update_layout(scene_camera=camera_switcher(hemi_, view),
                                 paper_bgcolor="rgba(0,0,0,0)",
                                 plot_bgcolor="rgba(0,0,0,0)")
        if colorbar: 
            mesh_data = fig.figure.data[1]
            mesh_data.update(colorbar=dict(
                len=0.5,
                tickfont=dict(color='black', size=25),
                tickformat='.',
                tickvals=tickvals,  # Example tick values
            ))
        fig.figure.write_image(f'{figure_prefix}_view-{view}_hemi-{hemi_}.png')


# Best in class
Plot the best performance in each voxel for each of the three classes of models

In [ ]:
for model_class, file in files.items():
    df = pd.read_csv(file).merge(metadata, on='voxel_id').set_index('subj_id')
    for subj_id, subj_df in tqdm(df.groupby('subj_id'), desc='Plotting subjects'):
        subj_id_str = str(subj_id).zfill(2)
        stat_file = f'{interim_dir}/{process}/sub-{subj_id_str}_class-{model_class}'
        plot_file = f'{figure_dir}/{process}/sub-{subj_id_str}_class-{model_class}'

        img = nib.load(f'{raw_dir}/fmri_betas/sub-{subj_id_str}_space-T1w_desc-train-fracridge_data.nii.gz')

        dims = img.shape[:-1]
        header, affine = img.header, img.affine

        scores_arr = subj_df['test_score'].to_numpy()
        indices = subj_df[['i_index', 'j_index', 'k_index']].to_numpy()

        score_img = np.zeros(dims)
        score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
        score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
        nib.save(score_img, f'{stat_file}.nii.gz')

        for hemi in ['lh', 'rh']:
            surf = compute_surf_stats(stat_file, subj_id_str, hemi)
            inflated, sulcus = load_surf_mesh(raw_dir, subj_id_str, hemi)
            plot_stats(inflated, sulcus, surf, hemi, plot_file)

# Vision - Language

In [38]:
if 'image' in files.keys():
    vision = pd.concat([pd.read_csv(files['image']), 
                       pd.read_csv(files['video'])])
    vision.reset_index(drop=True, inplace=True)
    idx = vision.groupby('voxel_id')['train_score'].idxmax()
    vision = vision.loc[idx].reset_index(drop=True)
else:
    vision = pd.read_csv(files['video'])

language = pd.read_csv(files['language'])
vision = vision[['voxel_id', 'test_score']].set_index('voxel_id').sort_index()
language = language[['voxel_id', 'test_score']].set_index('voxel_id').sort_index()
vision_language = vision - language
vision_language = vision_language.reset_index().merge(metadata, on='voxel_id')

In [ ]:
for subj_id, subj_df in tqdm(vision_language.groupby('subj_id'), desc='Plotting subjects'):
    subj_id_str = str(subj_id).zfill(2)
    stat_file = f'{interim_dir}/{process}/sub-{subj_id_str}_vision-language'
    plot_file = f'{figure_dir}/{process}/sub-{subj_id_str}_vision-language'

    img = nib.load(f'{raw_dir}/fmri_betas/sub-{subj_id_str}_space-T1w_desc-train-fracridge_data.nii.gz')

    dims = img.shape[:-1]
    header, affine = img.header, img.affine

    scores_arr = subj_df['test_score'].to_numpy()
    indices = subj_df[['i_index', 'j_index', 'k_index']].to_numpy()

    score_img = np.zeros(dims)
    score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
    score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
    nib.save(score_img, f'{stat_file}.nii.gz')

    for hemi in ['lh', 'rh']:
        surf = compute_surf_stats(stat_file, subj_id_str, hemi)
        inflated, sulcus = load_surf_mesh(raw_dir, subj_id_str, hemi)
        plot_stats(inflated, sulcus, surf, hemi, plot_file, vmax=.3, negative_values=True)

# Hiearchy

In [12]:
import numpy as np
from nilearn import plotting, surface
import nibabel as nib
from src.tools import camera_switcher


def compute_surf_stats(prefix, sub, hemi):
    file = f'{prefix}_hemi-{hemi}.mgz'
    if not os.path.exists(file):
        cmd = '/Applications/freesurfer/bin/mri_vol2surf '
        cmd += f'--src {prefix}.nii.gz '
        cmd += f'--out {file} '
        cmd += f'--regheader sub-{sub} '
        cmd += f'--hemi {hemi} '
        cmd += '--projfrac 1'
        os.system(cmd)
    return surface.load_surf_data(file)


def load_surf_mesh(path, sub, hemi):
    return f'{path}/freesurfer/sub-{sub}/surf/{hemi}.inflated', \
            f'{path}/freesurfer/sub-{sub}/surf/{hemi}.sulc'


def plot_stats(surf_mesh, bg_map, surf_map, hemi_, figure_prefix,
                vmax=0.6, negative_values=False,
                cmap_name='icefire', threshold=1e-6):
    cmap=sns.color_palette(cmap_name, as_cmap=True)
    hemi_name = 'left' if hemi_ == 'lh' else 'right'

    surf_map = np.nan_to_num(surf_map)
    if not negative_values:
        surf_map[surf_map < 0] = 0
        vmin = 0
        tickvals = list(np.round(np.linspace(vmin, vmax, num=5), 2)) 
    else:
        vmin = None
        tickvals = list(np.round(np.linspace(-1*vmax, vmax, num=5), 2)) 

    for view in ['ventral', 'lateral', 'medial']:
        colorbar = True if view == 'medial' and hemi_ == 'rh' else False
        fig = plotting.plot_surf_roi(surf_mesh=surf_mesh,
                                        roi_map=surf_map,
                                        bg_map=bg_map,
                                        threshold=threshold,
                                        vmax=vmax,
                                        vmin=vmin,
                                        engine='plotly',
                                        colorbar=colorbar,
                                        view=view,
                                        cmap=cmap,
                                        hemi=hemi_name)
        fig.figure.update_layout(scene_camera=camera_switcher(hemi_, view),
                                 paper_bgcolor="rgba(0,0,0,0)",
                                 plot_bgcolor="rgba(0,0,0,0)")
        if colorbar: 
            mesh_data = fig.figure.data[1]
            mesh_data.update(colorbar=dict(
                len=0.5,
                tickfont=dict(color='black', size=25),
                tickformat='.',
                tickvals=tickvals,  # Example tick values
            ))
        fig.figure.write_image(f'{figure_prefix}_view-{view}_hemi-{hemi_}.png')


In [40]:
if 'image' in files.keys():
    vision = pd.concat([pd.read_csv(files['image']), 
                       pd.read_csv(files['video'])])
    vision.reset_index(drop=True, inplace=True)
    idx = vision.groupby('voxel_id')['train_score'].idxmax()
    vision = vision.loc[idx].reset_index(drop=True)
else:
    vision = pd.read_csv(files['video'])
vision = vision.merge(metadata, on='voxel_id')

In [44]:
df = pd.read_csv(file).merge(metadata, on='voxel_id').set_index('subj_id')
for subj_id, subj_df in tqdm(df.groupby('subj_id'), desc='Plotting subjects'):
    subj_id_str = str(subj_id).zfill(2)
    stat_file = f'{interim_dir}/{process}/sub-{subj_id_str}_vision-hierarchy'
    plot_file = f'{figure_dir}/{process}/sub-{subj_id_str}_vision-hierarchy'

    img = nib.load(f'{raw_dir}/fmri_betas/sub-{subj_id_str}_space-T1w_desc-train-fracridge_data.nii.gz')

    dims = img.shape[:-1]
    header, affine = img.header, img.affine

    scores_arr = subj_df['layer_relative_depth'].to_numpy()
    indices = subj_df[['i_index', 'j_index', 'k_index']].to_numpy()

    score_img = np.zeros(dims)
    score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
    score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
    nib.save(score_img, f'{stat_file}.nii.gz')

    for hemi in ['lh', 'rh']:
        surf = compute_surf_stats(stat_file, subj_id_str, hemi)
        inflated, sulcus = load_surf_mesh(raw_dir, subj_id_str, hemi)
        plot_stats(inflated, sulcus, surf, hemi, plot_file, vmax=1., cmap_name='mako')

Plotting subjects:   0%|                                                                               | 0/4 [00:00<?, ?it/s]/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:85: UserWarning:

choosing both vmin and a threshold is not allowed; setting vmin to 0

Plotting subjects:  25%|█████████████████▊                                                     | 1/4 [00:27<01:22, 27.36s/it]/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:85: UserWarning:

choosing both vmin and a threshold is not allowed; setting vmin to 0



srcvol = /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/interim/WholeBrainPlotting/sub-02_vision-hierarchy.nii.gz
srcreg unspecified
srcregold = 0
srcwarp unspecified
surf = white
hemi = rh
ProjFrac = 1
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
Done loading volume
Computing registration from header.
  Using /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-02/mri/orig.mgz as target reference.
-------- original matrix -----------
-1.00000   0.00000   0.00000   2.23201;
 0.00000   0.00000   1.00000  -13.07619;
 0.00000  -1.00000   0.00000   1.95750;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-02/surf/rh.white
Done reading source surface
Reading thickness /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-02/surf/rh.thic

Plotting subjects:  50%|███████████████████████████████████▌                                   | 2/4 [00:53<00:53, 26.88s/it]

srcvol = /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/interim/WholeBrainPlotting/sub-03_vision-hierarchy.nii.gz
srcreg unspecified
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
ProjFrac = 1
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
Done loading volume
Computing registration from header.
  Using /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-03/mri/orig.mgz as target reference.
-------- original matrix -----------
-1.00000   0.00000   0.00000  -1.48973;
 0.00000   0.00000   1.00000   5.51627;
-0.00000  -1.00000  -0.00000  -4.88640;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-03/surf/lh.white
Done reading source surface
Reading thickness /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-03/surf/lh.thick

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:85: UserWarning:

choosing both vmin and a threshold is not allowed; setting vmin to 0



...

Plotting subjects:  75%|█████████████████████████████████████████████████████▎                 | 3/4 [01:20<00:26, 26.81s/it]

srcvol = /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/interim/WholeBrainPlotting/sub-04_vision-hierarchy.nii.gz
srcreg unspecified
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
ProjFrac = 1
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
Done loading volume
Computing registration from header.
  Using /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/mri/orig.mgz as target reference.
-------- original matrix -----------
-1.00000  -0.00000  -0.00000   0.61232;
 0.00000   0.00000   1.00000   29.66757;
 0.00000  -1.00000   0.00000   1.95782;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/surf/lh.white
Done reading source surface
Reading thickness /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/surf/lh.thic

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:85: UserWarning:

choosing both vmin and a threshold is not allowed; setting vmin to 0



srcvol = /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/interim/WholeBrainPlotting/sub-04_vision-hierarchy.nii.gz
srcreg unspecified
srcregold = 0
srcwarp unspecified
surf = white
hemi = rh
ProjFrac = 1
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
Done loading volume
Computing registration from header.
  Using /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/mri/orig.mgz as target reference.
-------- original matrix -----------
-1.00000  -0.00000  -0.00000   0.61232;
 0.00000   0.00000   1.00000   29.66757;
 0.00000  -1.00000   0.00000   1.95782;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/surf/rh.white
Done reading source surface
Reading thickness /Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling/data/raw/freesurfer/sub-04/surf/rh.thic

Plotting subjects: 100%|███████████████████████████████████████████████████████████████████████| 4/4 [01:50<00:00, 27.69s/it]


In [ ]:
for model_class, file in files.items():
    df = pd.read_csv(file).merge(metadata, on='voxel_id').set_index('subj_id')
    for subj_id, subj_df in tqdm(df.groupby('subj_id'), desc='Plotting subjects'):
        subj_id_str = str(subj_id).zfill(2)
        stat_file = f'{interim_dir}/{process}/sub-{subj_id_str}_class-{model_class}'
        plot_file = f'{figure_dir}/{process}/sub-{subj_id_str}_class-{model_class}'

        img = nib.load(f'{raw_dir}/fmri_betas/sub-{subj_id_str}_space-T1w_desc-train-fracridge_data.nii.gz')

        dims = img.shape[:-1]
        header, affine = img.header, img.affine

        scores_arr = subj_df['test_score'].to_numpy()
        indices = subj_df[['i_index', 'j_index', 'k_index']].to_numpy()

        score_img = np.zeros(dims)
        score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
        score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
        nib.save(score_img, f'{stat_file}.nii.gz')

        for hemi in ['lh', 'rh']:
            surf = compute_surf_stats(stat_file, subj_id_str, hemi)
            inflated, sulcus = load_surf_mesh(raw_dir, subj_id_str, hemi)
            plot_stats(inflated, sulcus, surf, hemi, plot_file)

In [13]:
files = ['../data/interim/ResultsSummary/vision-language_whole-brain.csv.gz']
df = pd.read_csv(file).merge(metadata, on='voxel_id').set_index('subj_id')
for subj_id, subj_df in tqdm(df.groupby('subj_id'), desc='Plotting subjects'):
    subj_id_str = str(subj_id).zfill(2)
    stat_file = f'{interim_dir}/{process}/sub-{subj_id_str}_vision-language'
    plot_file = f'{figure_dir}/{process}/sub-{subj_id_str}_vision-language'

    img = nib.load(f'{raw_dir}/fmri_betas/sub-{subj_id_str}_space-T1w_desc-train-fracridge_data.nii.gz')

    dims = img.shape[:-1]
    header, affine = img.header, img.affine

    scores_arr = subj_df['test_score'].to_numpy()
    indices = subj_df[['i_index', 'j_index', 'k_index']].to_numpy()

    score_img = np.zeros(dims)
    score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
    score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
    nib.save(score_img, f'{stat_file}.nii.gz')

    for hemi in ['lh', 'rh']:
        surf = compute_surf_stats(stat_file, subj_id_str, hemi)
        inflated, sulcus = load_surf_mesh(raw_dir, subj_id_str, hemi)
        plot_stats(inflated, sulcus, surf, hemi, plot_file, vmax=.3, negative_values=True)

Plotting subjects:   0%|                                                                               | 0/4 [00:00<?, ?it/s]/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:76: UserWarning:

you have specified symmetric_cmap=False but the map contains negative values; setting symmetric_cmap to True

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:81: UserWarning:

vmin cannot be chosen when cmap is symmetric

Plotting subjects:  25%|█████████████████▊                                                     | 1/4 [00:26<01:18, 26.05s/it]/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:76: UserWarning:

you have specified symmetric_cmap=False but the map contains negative values; setting symmetric_cmap to True

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/nilearn/plotting/js_plotting_utils.py:81: UserWarnin

KeyboardInterrupt: 